# Algoritmo branch-and-bound

---

Este projeto consiste na criação de um programa para resolver problemas de maximização em programação linear inteira binária. Utilizaremos o algoritmo Branch-and-Bound para encontrar a solução ótima. 

O objetivo é encontrar o valor máximo de uma função sujeita a diversas restrições, todas do tipo "menor ou igual". 

O algoritmo irá dividir o problema em subproblemas menores e, através de uma estratégia de ramificação, explorará diferentes soluções até encontrar a ótima. A Estratégia de ramificação usada será a: <br> <br>

<div style="border:2px solid black; padding: 10px; background-color:#CEE0DC">
Busca em profundidade: Para isso, utilizamos uma pilha para armazenar os nós abertos (nós que ainda não foram processados)</div>

Para resolver os subproblemas, faremos uso de um pacote de programação linear, como o python-mip. 

---

In [3]:
from mip import *

In [4]:
# Função para ler os dados do arquivo
def ler_arquivo(arquivo):
    with open(arquivo, 'r') as f:
        
        num_vars, num_rest = map(int, f.readline().split()) # Lê o número de variáveis e o número de restrições
        coef_obj = list(map(float, f.readline().split())) # Lê os coeficientes da função objetivo
        coef_rest = [] # Inicializa a lista de restrições
        
        # Lê as restrições e o lado direito das restrições
        for _ in range(num_rest):
            linha = list(map(float, f.readline().split()))
            coef_rest.append(linha[:-1])  # Coeficientes da restrição
            coef_rest[-1].append(linha[-1])  # Lado direito da restrição
            
    return num_vars, coef_obj, coef_rest

Exemplo teste1:

$$
\text{max } Z = 2x_1 + 10x_2 + 8x_3 + 7x_4 + 10x_5 + 10x_6 + 6x_7
$$

s.a. 
$$
\begin{gather*}
5x_1 + 7x_2 + 8x_3 + 1x_4 + 7x_5 + 5x_6 + 6x_7\leq 20 \\
1x_1 + 6x_2 + 4x_3 + 9x_4 + 10x_5 + 6x_6 + 10x_7 \leq 30 \\
4x_1 + 4x_2 + 4x_3 + 1x_4 + 5x_5 + 5x_6 + 10x_7 \leq 40 \\
3x_1 + 10x_2 + 8x_3 + 1x_4 + 3x_5 + 3x_6 + 8x_7\leq 30 \\
10x_1 + 8x_2 + 9x_3 + 9x_4 + 7x_5 + 6x_6 + 10x_7\leq 20 \\
6x_1 + 6x_2 + 3x_3 + 6x_4 + 3x_5 + 7x_6 + 2x_7\leq 80 \\
7x_1 + 10x_2 + 7x_3 + 8x_4 + 7x_5 + 8x_6 + 7x_7\leq 100 \\
9x_1 + 8x_2 + 1x_3 + 1x_4 + 8x_5 + 10x_6 + 2x_7\leq 90 \\
1x_1 + 5x_2 + 3x_3 + 10x_4 + 2x_5 + 4x_6 + 9x_7\leq 70 \\
9x_1 + 6x_2 + 1x_3 + 4x_4 + 7x_5 + 5x_6 + 10x_7\leq 60 \\
5x_1 + 7x_2 + 4x_3 + 4x_4 + 3x_5 + 4x_6 + 10x_7\leq 40 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0, 1\}
\end{gather*}
$$

In [5]:
class No: #Nó da arvore
    def __init__(self, solucao_relaxacao, limite_inferior_objetivo, limite_superior_objetivo, vars_inteiras, restricoes):
        self.solucao_relaxacao = solucao_relaxacao
        self.limite_inferior_objetivo = limite_inferior_objetivo
        self.limite_superior_objetivo = limite_superior_objetivo
        self.vars_inteiras = vars_inteiras
        self.restricoes = restricoes

In [6]:
def construir_modelo(num_vars, coef_obj, coef_rest):
    # Cria um modelo de maximização
    modelo = Model(sense=MAXIMIZE, solver_name=CBC)

    # Adiciona as variáveis de decisão ao modelo
    x = [modelo.add_var(var_type=BINARY) for _ in range(num_vars)]

    # Define a função objetivo
    modelo.objective = maximize(xsum(coef_obj[i] * x[i] for i in range(num_vars)))

    # Adiciona as restrições ao modelo
    for i in range(len(coef_rest)):
        modelo += xsum(coef_rest[i][j] * x[j] for j in range(num_vars)) <= coef_rest[i][-1]
    print(modelo)
    return modelo

Exemplo teste1:

$$
\text{max } Z = 2x_1 + 10x_2 + 8x_3 + 7x_4 + 10x_5 + 10x_6 + 6x_7
$$

$$
\begin{gather*}
5x_1 + 7x_2 + 8x_3 + 1x_4 + 7x_5 + 5x_6 + 6x_7\leq 20 \\
1x_1 + 6x_2 + 4x_3 + 9x_4 + 10x_5 + 6x_6 + 10x_7 \leq 30 \\
4x_1 + 4x_2 + 4x_3 + 1x_4 + 5x_5 + 5x_6 + 10x_7 \leq 40 \\
3x_1 + 10x_2 + 8x_3 + 1x_4 + 3x_5 + 3x_6 + 8x_7\leq 30 \\
10x_1 + 8x_2 + 9x_3 + 9x_4 + 7x_5 + 6x_6 + 10x_7\leq 20 \\
6x_1 + 6x_2 + 3x_3 + 6x_4 + 3x_5 + 7x_6 + 2x_7\leq 80 \\
7x_1 + 10x_2 + 7x_3 + 8x_4 + 7x_5 + 8x_6 + 7x_7\leq 100 \\
9x_1 + 8x_2 + 1x_3 + 1x_4 + 8x_5 + 10x_6 + 2x_7\leq 90 \\
1x_1 + 5x_2 + 3x_3 + 10x_4 + 2x_5 + 4x_6 + 9x_7\leq 70 \\
9x_1 + 6x_2 + 1x_3 + 4x_4 + 7x_5 + 5x_6 + 10x_7\leq 60 \\
5x_1 + 7x_2 + 4x_3 + 4x_4 + 3x_5 + 4x_6 + 10x_7\leq 40 \\
x_1, x_2, x_3, x_4, x_5, x_6, x_7 \geq 0 \\
\end{gather*}
$$
<div style="text-align: center; color: red;"><s>$x_1, x_2, x_3, x_4, x_5, x_6, x_7 \in \{0, 1\}$</s></div>


In [15]:
def branch_and_bound(modelo):
    
    # os valores para solução iniciais são infinitos
    nos = [No(None, float('-inf'), float('inf'), modelo.vars, [])]
    melhor_solucao = float('-inf')

    while nos:
        no = nos.pop()  # Seleciona nó da pilha

        # Resolve a relaxação do nó atual
        solucao_relaxacao = no.solucao_relaxacao
        
        # verificamos se já temos uma solução relaxada disponível para o nó atual 
        if solucao_relaxacao is None:
            m = modelo.copy()
        else:
            m = modelo.copy(solucao_relaxacao)

        m.optimize() # relaxação do nó atual (limite dual atual - inferior)

        #verificando se o solver encontrou alguma solução para o problema relaxado.
        if m.num_solutions:
            valor_objetivo = m.objective_value
            if valor_objetivo > melhor_solucao:
                melhor_solucao = valor_objetivo
                print(f"\nMelhor solução encontrada:\033[1;31;47m {melhor_solucao} \033[0m\n")


        # Verifica se o nó atual pode ser podado - INVIABILIDADE OU LIMITANTE
        if m.num_solutions == 0 or valor_objetivo <= no.limite_inferior_objetivo:
            continue

        # Verifica se a solução da relaxação é inteira - INTEGRALIDADE
        solucao_inteira = all(int(sol + 0.5) == sol for sol in [var.x for var in m.vars])
        if solucao_inteira:
            continue

        # Escolhe a variável para ramificar (mais próxima de 0.5)
        vars_fracionarias = [(var, abs(sol - 0.5)) for var, sol in zip(m.vars, [var.x for var in m.vars]) if not int(sol + 0.5) == sol]
        vars_fracionarias.sort(key=lambda x: x[1], reverse=True)
        var_para_ramificar, _ = vars_fracionarias[0]

        # Ramifica na variável
        for val in [0, 1]:
            novas_restricoes = no.restricoes + [var_para_ramificar == val]
            novo_modelo = m.copy(no.solucao_relaxacao)
            for restricao in novas_restricoes:
                novo_modelo += restricao

            novo_no = No(None, no.limite_inferior_objetivo, no.limite_superior_objetivo, modelo.vars, novas_restricoes)
            nos.append(novo_no)

In [16]:
# Importa os dados do arquivo
nome_arquivo = "teste1.txt"
num_vars, coef_obj, coef_rest = ler_arquivo(nome_arquivo)

# Constrói o modelo
modelo = construir_modelo(num_vars, coef_obj, coef_rest)

# Resolve o problema utilizando branch-and-bound
solucao = branch_and_bound(modelo)

modelo.write("modelo.lp") # Salva o modelo em um arquivo
with open("modelo.lp", "r") as f: # Lê e exibe o conteúdo do arquivo
    print(f.read())


Melhor solução encontrada: 20.0 

\Problem name: 

Minimize
OBJROW: -2 var(0) -10 var(1) -8 var(2) -7 var(3) -10 var(4) -10 var(5) -6 var(6)
Subject To
constr(0):  5 var(0) + 7 var(1) + 8 var(2) + var(3) + 7 var(4) + 5 var(5) + 6 var(6) <= 20
constr(1):  var(0) + 6 var(1) + 4 var(2) + 9 var(3) + 10 var(4) + 6 var(5) + 10 var(6) <= 30
constr(2):  4 var(0) + 4 var(1) + 4 var(2) + var(3) + 5 var(4) + 5 var(5) + 10 var(6) <= 40
constr(3):  3 var(0) + 10 var(1) + 8 var(2) + var(3) + 3 var(4) + 3 var(5) + 8 var(6) <= 30
constr(4):  10 var(0) + 8 var(1) + 9 var(2) + 9 var(3) + 7 var(4) + 6 var(5) + 10 var(6) <= 20
constr(5):  6 var(0) + 6 var(1) + 3 var(2) + 6 var(3) + 3 var(4) + 7 var(5) + 2 var(6) <= 80
constr(6):  7 var(0) + 10 var(1) + 7 var(2) + 8 var(3) + 7 var(4) + 8 var(5) + 7 var(6) <= 100
constr(7):  9 var(0) + 8 var(1) + var(2) + var(3) + 8 var(4) + 10 var(5) + 2 var(6) <= 90
constr(8):  var(0) + 5 var(1) + 3 var(2) + 10 var(3) + 2 var(4) + 4 var(5) + 9 var(6) <= 70
constr(9):  9 

In [17]:
nome_arquivo = "teste2.txt"
num_vars, coef_obj, coef_rest = ler_arquivo(nome_arquivo)
modelo = construir_modelo(num_vars, coef_obj, coef_rest)
solucao = branch_and_bound(modelo)

modelo.write("modelo.lp") # Salva o modelo em um arquivo
with open("modelo.lp", "r") as f: # Lê e exibe o conteúdo do arquivo
    print(f.read())


Melhor solução encontrada: 24.0 

\Problem name: 

Minimize
OBJROW: -7 var(0) -7 var(1) -7 var(2) -5 var(3) -8 var(4) -8 var(5) -9 var(6) -10 var(7) -7 var(8)
Subject To
constr(0):  var(0) + 3 var(1) + var(2) + 3 var(3) + 3 var(4) + 7 var(5) + 2 var(6) + var(7) + 4 var(8) <= 80
constr(1):  7 var(0) + 6 var(1) + 10 var(2) + var(3) + 7 var(4) + 2 var(5) + 2 var(6) + 7 var(7) + var(8) <= 90
constr(2):  3 var(0) + 2 var(1) + var(2) + 3 var(3) + 3 var(4) + 2 var(5) + var(6) + 6 var(7) + 5 var(8) <= 10
constr(3):  10 var(0) + 8 var(1) + 3 var(2) + 6 var(3) + 10 var(4) + 7 var(5) + 3 var(6) + 10 var(7) + 4 var(8) <= 30
constr(4):  2 var(0) + 8 var(1) + 6 var(2) + 5 var(3) + 6 var(4) + 6 var(5) + 9 var(6) + 7 var(7) + 2 var(8) <= 80
constr(5):  3 var(0) + 10 var(1) + var(2) + 9 var(3) + 2 var(4) + 7 var(5) + 7 var(6) + 9 var(7) + 10 var(8) <= 90
constr(6):  var(0) + 7 var(1) + 10 var(2) + 10 var(3) + 5 var(4) + 2 var(5) + 9 var(6) + 10 var(7) + 2 var(8) <= 20
constr(7):  10 var(0) + 3 var(1) 

In [18]:
nome_arquivo = "teste3.txt"
num_vars, coef_obj, coef_rest = ler_arquivo(nome_arquivo)
modelo = construir_modelo(num_vars, coef_obj, coef_rest)
solucao = branch_and_bound(modelo)

modelo.write("modelo.lp") # Salva o modelo em um arquivo
with open("modelo.lp", "r") as f: # Lê e exibe o conteúdo do arquivo
    print(f.read())


Melhor solução encontrada: 19.0 

\Problem name: 

Minimize
OBJROW: -7 var(0) -9 var(1) -10 var(2) -3 var(3) -6 var(4) - var(5) -9 var(6) -8 var(7) -8 var(8)
Subject To
constr(0):  2 var(0) + var(1) + 9 var(2) + 6 var(3) + 3 var(4) + 6 var(5) + 10 var(6) + 9 var(7) + var(8) <= 60
constr(1):  8 var(0) + 6 var(1) + 6 var(2) + 5 var(3) + 2 var(4) + 2 var(5) + 4 var(6) + 3 var(7) + 6 var(8) <= 80
constr(2):  8 var(0) + var(1) + 3 var(2) + 7 var(3) + var(4) + 4 var(5) + 8 var(6) + 3 var(7) + 4 var(8) <= 30
constr(3):  6 var(0) + 3 var(1) + 9 var(2) + 5 var(3) + 9 var(4) + 6 var(5) + 9 var(6) + 9 var(7) + 6 var(8) <= 40
constr(4):  10 var(0) + 8 var(1) + 8 var(2) + 7 var(3) + 10 var(4) + 10 var(5) + 9 var(6) + 9 var(7) + 3 var(8) <= 20
constr(5):  10 var(0) + 10 var(1) + 10 var(2) + 9 var(3) + 10 var(4) + var(5) + 8 var(6) + 3 var(7) + 10 var(8) <= 90
constr(6):  10 var(0) + 5 var(1) + 8 var(2) + 2 var(3) + 7 var(4) + 8 var(5) + 6 var(6) + 2 var(7) + 2 var(8) <= 90
constr(7):  7 var(0) + 9 

In [19]:
nome_arquivo = "teste4.txt"
num_vars, coef_obj, coef_rest = ler_arquivo(nome_arquivo)
modelo = construir_modelo(num_vars, coef_obj, coef_rest)
solucao = branch_and_bound(modelo)

modelo.write("modelo.lp") # Salva o modelo em um arquivo
with open("modelo.lp", "r") as f: # Lê e exibe o conteúdo do arquivo
    print(f.read())


Melhor solução encontrada: 10.0 

\Problem name: 

Minimize
OBJROW: -9 var(0) -7 var(1) -10 var(2) -7 var(3) -9 var(4) -6 var(5) -8 var(6) -4 var(7) -9 var(8)
Subject To
constr(0):  4 var(0) + 9 var(1) + 4 var(2) + var(3) + 9 var(4) + 6 var(5) + 3 var(6) + 6 var(7) + var(8) <= 40
constr(1):  3 var(0) + 7 var(1) + 8 var(2) + 7 var(3) + 6 var(4) + 3 var(5) + 5 var(6) + 9 var(7) + 4 var(8) <= 80
constr(2):  9 var(0) + 3 var(1) + 6 var(2) + 5 var(3) + 7 var(4) + var(5) + var(6) + 3 var(7) + 9 var(8) <= 40
constr(3):  5 var(0) + 9 var(1) + 6 var(2) + 5 var(3) + 9 var(4) + 7 var(5) + 8 var(6) + 7 var(7) + 8 var(8) <= 10
constr(4):  7 var(0) + 7 var(1) + 4 var(2) + var(3) + 3 var(4) + 4 var(5) + 8 var(6) + var(7) + 9 var(8) <= 10
constr(5):  var(0) + 6 var(1) + 6 var(2) + var(3) + 6 var(4) + 7 var(5) + 3 var(6) + 8 var(7) + 7 var(8) <= 10
constr(6):  6 var(0) + 6 var(1) + 8 var(2) + 6 var(3) + 10 var(4) + 8 var(5) + var(6) + 4 var(7) + 4 var(8) <= 70
constr(7):  9 var(0) + var(1) + 9 var(2) 